In [1]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
bike_speed = 10
fly_speed = 15
start_distance = 20

In [3]:
class Mover():
    def __init__(self, pos, velocity, t=0):
        self.velocity = velocity
        self.set_reference_position(pos, t)
        
    def set_reference_position(self, pos, t=0):
        self.init_pos = pos - t * self.velocity
        
    def position(self, t):
        return self.init_pos + self.velocity * t
    
    def collision_time(self, other):
        if abs(self.velocity - other.velocity) == 0.0:
            return None
        return (other.init_pos - self.init_pos) / (self.velocity - other.velocity)
    
    def turn_around_at(self, t):
        pos = self.position(t)
        self.velocity = -self.velocity
        self.set_reference_position(pos, t)        

In [21]:
# initialize the bike objects
bike1 = Mover(pos=0, velocity=bike_speed)
bike2 = Mover(pos=start_distance, velocity=-bike_speed)

# initialize the fly object at T=0
t = 0
fly = Mover(pos=bike1.position(t), velocity=fly_speed)

# Keep track of the fly's position at the times when its velocity changes
fly_position = [(t, fly.position(t))]

# ...and the total distance it has flown
total_distance = 0.0
print("{:7s}  {:7s} {:7s} {:7s} {:8s} {:7s}".format(
    "time", " bike1", " fly", " bike2", "distance", " total"))
print("{:7.5f}  {:7.4f} {:7.4f} {:7.4f}  {:7.4f} {:7.4f}".format(
    t, bike1.position(t), fly.position(t), bike2.position(t), 0, total_distance))

while len(fly_position) < 10:
    # find the time when the fly next encounters a bike.
    t1 = fly.collision_time(bike1)
    t2 = fly.collision_time(bike2)
    # either t1 or t2 should be essentially equal to t (when the fly last 
    # changed direction) and the other time should be greater.  We'll just 
    # assume the later of the two times is when the fly next encounters a bike.
    if t1 > t2:
        turn_t = t1
    else:
        turn_t = t2
    distance = abs(fly.position(turn_t) - fly.position(t))
    fly_position.append((turn_t, fly.position(turn_t)))
    total_distance +=  distance
    fly.turn_around_at(turn_t)
    
    t = turn_t
    print("{:7.5f}  {:7.4f} {:7.4f} {:7.4f}  {:7.4f} {:7.4f}".format(
        t, bike1.position(t), fly.position(t), bike2.position(t), distance, total_distance))

time      bike1   fly     bike2  distance  total 
0.00000   0.0000  0.0000 20.0000   0.0000  0.0000
0.80000   8.0000 12.0000 12.0000  12.0000 12.0000
0.96000   9.6000  9.6000 10.4000   2.4000 14.4000
0.99200   9.9200 10.0800 10.0800   0.4800 14.8800
0.99840   9.9840  9.9840 10.0160   0.0960 14.9760
0.99968   9.9968 10.0032 10.0032   0.0192 14.9952
0.99994   9.9994  9.9994 10.0006   0.0038 14.9990
0.99999   9.9999 10.0001 10.0001   0.0008 14.9998
1.00000  10.0000 10.0000 10.0000   0.0002 15.0000
1.00000  10.0000 10.0000 10.0000   0.0000 15.0000


In [23]:
t = [item[0] for item in fly_position]
pfly = [item[1] for item in fly_position]
pbike1 = [bike1.position(ti) for ti in t]
pbike2 = [bike2.position(ti) for ti in t]

fig = figure(title="The Bikes and the Fly",
             x_axis_label='time (hours)',
             y_axis_label='position (miles)')

# add a line renderer with legend and line thickness
fig.line(t, pfly, legend="fly", line_width=2, line_color="red")
fig.line(t, pbike1, legend="bike1", line_width=2, line_dash="2 2", line_color="lightblue")
fig.line(t, pbike2, legend="bike2", line_width=2, line_dash="2 2", line_color="gray")

# show the results
show(fig)